<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/5.%20RunnableLambda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. `RunnableLambda`**
---
`RunnableLambda` es un componente clave en LangChain Expression Language (LCEL) que permite
integrar funciones de Python estándar directamente en tus cadenas y flujos de trabajo.
En esencia, `RunnableLambda` toma una función de Python y la convierte en un objeto
"runnable" que puede ser parte de una cadena de LangChain, recibiendo la entrada del
paso anterior y pasando su salida al siguiente.

**¿Por qué usar RunnableLambda?**

* **Flexibilidad:** Integra lógica de Python arbitraria dentro de tus cadenas de LangChain.  

* **Simplicidad:** Envuelve funciones existentes sin necesidad de crear clases complejas.
* **Modularidad:** Permite descomponer tareas complejas en funciones más pequeñas y reutilizables.
* **Integración:** Facilita la conexión de LangChain con herramientas y librerías de Python.

Vamos a ver a continuación varios ejemplos de uso

# **Preparando el entorno del cuaderno**
---
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google Hugging Face, Mistral, Together y Anthropic

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.
El uso de las API de OpenAI y Anthropic es de pago. El resto son gratuitas y para usarlas basta con registrarse y generar una API Key.  

En el primer cuaderno encontraras los enlaces a estos servicios y este codigo explicado

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic


# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

## **Ejemplo 1: Envolviendo funciones simples**
---

La forma principal de añadir una función a una cadena en Langchain (especialmente en LCEL) es utilizando RunnableLambda. RunnableLambda es un Runnable que envuelve una función Python, permitiéndole integrarse perfectamente en el flujo de la cadena.

(Aunque es posible, la inserción directa de una función en una cadena funciona, esto se debe a que Langchain implícitamente envuelve esa función en un RunnableLambda "por debajo del capó" para que pueda encajar dentro del paradigma de la cadena LCEL. Es mejor ser consistente con los principios del framework y usar RunnableLambda)

In [ ]:
from langchain_core.runnables import RunnableLambda

# Función simple de Python
def duplicar(x):
    # Duplica el valor de entrada
    return x * 2

# Convertir a Runnable
runnable_duplicar = RunnableLambda(duplicar)

# Invocar el Runnable
resultado = runnable_duplicar.invoke(5)
print(f"Resultado de duplicar: {resultado}")  # Salida: 10

## **Ejemplo 2: Usando RunnableLambda en una cadena simple**
---

Múltiples `RunnableLambda` en secuencia. Este ejemplo muestra cómo encadenar múltiples `RunnableLambda`. La salida del primero (`runnable_mayusculas`) se convierte en la entrada del segundo
(`runnable_exclamacion`).

In [ ]:
from langchain_core.runnables import RunnableLambda

def mayusculas(texto):
    return texto.upper()

def agregar_exclamacion(texto):
    return texto + "!"

runnable_mayusculas = RunnableLambda(mayusculas)
runnable_exclamacion = RunnableLambda(agregar_exclamacion)

cadena_transformacion = runnable_mayusculas | runnable_exclamacion

resultado_transformacion = cadena_transformacion.invoke("Este es un texto cualquiera")
print(resultado_transformacion)

## **Ejemplo 3: Preprocesamiento del prompt**

Usamos un `RunnableLambda` con una función que se encargara de "limpiar" el texto que vamos a enviar para contruir el prompt template.



In [ ]:
from langchain_core.runnables import RunnableLambda

def limpiar_texto(texto):
    """Elimina espacios extra y convierte a minúsculas"""
    return texto.strip().lower()

# Crear un flujo simple de procesamiento
prompt_template = PromptTemplate.from_template(
    "Analiza el siguiente texto: {texto}"
)

print(type(prompt_template))


cadena_procesamiento = RunnableLambda(limpiar_texto) | prompt_template  # Primero limpia, luego aplica plantilla


# Demostración
print("\nFlujo de procesamiento de texto:")
resultado = cadena_procesamiento.invoke("  EJEMPLO de Texto  ")
print(resultado)

## **Ejemplo 4: Transformacion de datos entre Runnables**

Un caso de uso habitual de RunnableLambda es transformar los datosentre los pasos de una cadena de LangChain. Veamos cómo podemos usarlo para conseguir una lista a partir de un diccionario


In [ ]:
from langchain_core.runnables import RunnableLambda

# Paso 1: Crear un RunnableLambda para extraer la lista de nombres.
# Usamos list comprehension https://ellibrodepython.com/list-comprehension-python
extract_names = RunnableLambda(
    lambda data: [person["name"] for person in data["people"]]
)

# Paso 2: Probar la cadena con un diccionario de ejemplo.
input_data = {
    "people": [
        {"name": "Juan", "age": 30},
        {"name": "Ana", "age": 25},
        {"name": "Luis", "age": 40}
    ]
}
result = extract_names.invoke(input_data)

# Imprimir el resultado final.
print(result)
print(type(result))

## **Ejemplo 5: Filtrado de datos**

Uno de los usos más poderosos de `RunnableLambda` es filtar los datos entre los pasos de una cadena de LangChain. Veamos cómo podemos usarlo para preparar la entrada de un `PromptTemplate`.

Imaginemos un caso de uso más realista donde necesitamos preprocesar datos antes de enviarlos a un LLM. Supongamos que tenemos una lista de productos y queremos filtrar aquellos que tienen un precio superior a un cierto valor.

En este ejemplo, `RunnableLambda` actúa como una etapa de preprocesamiento en nuestro flujo.
Podríamos integrar esto en una cadena más larga donde, por ejemplo, se le pide a un LLM que genere una descripción de estos productos caros.

In [ ]:
from langchain_core.runnables import RunnableLambda


productos = [
    {"nombre": "Laptop", "precio": 1200},
    {"nombre": "Teclado", "precio": 75},
    {"nombre": "Monitor", "precio": 300},
    {"nombre": "Ratón", "precio": 25},
]

# Creamos una funcion que recibe una lista de productos y un precio minimo
# y devuelve otra lista, filtrada
def filtrar_productos_caros(productos: list[dict], precio_minimo: int) -> list[dict]:
  """Filtra productos cuyo precio es mayor o igual al precio mínimo."""
  return [producto for producto in productos if producto["precio"] >= precio_minimo]

# Construimos un runnable con nuestra funcion de filtro
filtro_productos_runnable = RunnableLambda(
    lambda x: filtrar_productos_caros(x, precio_minimo=100)
)

productos_caros = filtro_productos_runnable.invoke(productos)

print("Lista de productos original:")
for producto in productos:
  print(f"- {producto['nombre']} (Precio: {producto['precio']})")

print("\nProductos con precio mayor o igual a 100:")
for producto in productos_caros:
  print(f"- {producto['nombre']} (Precio: {producto['precio']})")

## **Ejemplo 6: Postprocesamiento**

Otro patron de uso frecuente es validar la salida de un LLM antes de ser mostrada al usuario.

Supongamos para este ejemplo que tenemos una lista de palabras prohibidas. Si la respuesta del LLM incluye alguna de ellas, la salida no debe mostrarse

In [ ]:
from langchain_core.runnables import RunnableLambda


forbidden_words = ["prohibido", "secreto", "inadecuado"]  # Ejemplo

llm_gpt4o_mini = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=1)

def validate_output(aimessage):
    text=aimessage.content
    if any(word in text.lower() for word in forbidden_words):
        return "⚠️ Se ha detectado contenido prohibido. Operación detenida."
    else:
        return text


cadena = llm_gpt4o_mini | RunnableLambda(validate_output)

# Esto no deberia producir una respuesta con palabras prohibidas
prompt = "Escribe una breve frase motivacional sobre el aprendizaje."
respuesta = cadena.invoke(prompt)
print("Prompt:", prompt)
print("Respuesta:", respuesta)

# Esto SI deberia producir una respuesta con palabras prohibidas
prompt = "Menciona algo 'prohibido' que la gente suele hacer en secreto."
respuesta = cadena.invoke(prompt)
print("Prompt:", prompt)
print("Respuesta:", respuesta)




## **Desafio: Generador de TagLine**

Supongamos que disponemos de una paso anterior de un diccionario con información de una pelicula con el titulo, el año y una breve descripcion.

El objetivo es convertir este diccionario en un texto resumido que luego inyectaremos en un PromptTemplate. Por ejemplo, podemos combinar todos los campos y formatearlos en un breve párrafo.

El prompt template tiene que consegiur que el LLM genere una Tagline llamativo

In [ ]:
from langchain_core.runnables import RunnableLambda

movie_data = {
    "title": "The Matrix",
    "year": 1999,
    "description": "Thomas Anderson lleva una doble vida: por el día es programador en una importante empresa de software, y por la noche un hacker informático llamado Neo. Su vida no volverá a ser igual cuando unos misteriosos personajes le inviten a descubrir la respuesta a la pregunta que no le deja dormir: ¿qué es Matrix?"
}

# Esta funcion recibe el diccionario y devuleve un string
def transform_data(data):
    return f"Title: {data['title']}\nYear: {data['year']}\nDescription: {data['description']}"


llm_gpt4o_mini = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=1)

tagline_template = PromptTemplate(
    input_variables=["movie_info"],
    template=(
        "Eres un redactor creativo. Usa la información de la película a continuación para redactar 5 tagline "
        "cortos, emocionantes y fáciles de recordar:\n\n"
        "{movie_info}\n\n"
        "Tagline:"
    )
)

cadena = RunnableLambda(transform_data) | tagline_template | llm_gpt4o_mini

tagline = cadena.invoke(movie_data)
print(tagline.content)
